# Overview

1. [__Importing data__](#1)
    1. [Loading csv files](#1.1)


2. [__Preprocessing data__](#2)
    1. [Droping NaNs](#2.1)
    2. [Cleaning dataset](#2.2)
    3. [Datetime](#2.3)
    4. [Selecting data](#2.4)


3. [__Trial plots__](#3)
    1. [Number of protection plans activated per Catalonia regions](#3.1)
        1. [Províncies](#3.1.1)
        2. [Comarques](#3.1.2)
        3. [Municipis](#3.1.3)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# 1. Importing data <a class="anchor" id="1"></a>

## 1.1. Loading csv files <a class="anchor" id="1.1"></a>

In [2]:
data_dir = "dades/"

rescue_actions = pd.read_csv(data_dir+"Actuacions_en_salvaments_al_medi_natural_dels_Bombers_de_la_Generalitat_20231110.csv")
protection_plans = pd.read_csv(data_dir+"Registre_general_de_plans_de_protecci__civil_de_Catalunya_20231110.csv")
inc_112 = pd.read_csv(data_dir+"Dades_d_incidents_operatius_gestionats_pel_CAT112_20231110.csv")
calls_112 = pd.read_csv(data_dir+"Dades_de_trucades_operatives_gestionades_pel_CAT112_20231110.csv")


https://analisi.transparenciacatalunya.cat/Seguretat/Actuacions-en-salvaments-al-medi-natural-dels-Bomb/fsum-2k6e
https://analisi.transparenciacatalunya.cat/Seguretat/Registre-general-de-plans-de-protecci-civil-de-Cat/xqqe-tgav
https://analisi.transparenciacatalunya.cat/Seguretat/Dades-d-incidents-operatius-gestionats-pel-CAT112/jq8m-d7cw
https://analisi.transparenciacatalunya.cat/Seguretat/Dades-de-trucades-operatives-gestionades-pel-CAT11/mfqb-sbx4

In [3]:
comarques = ["Alt Camp","Alt Empordà","Alt Penedès","Alt Urgell","Alta Ribagorça","Anoia","Aran","Bages","Baix Camp",
             "Baix Ebre","Baix Empordà","Baix Llobregat","Baix Penedès","Barcelonès","Berguedà","Cerdanya",
             "Conca de Barberà","Garraf","Garrigues","Garrotxa","Gironès","Maresme","Moianès","Montsià",
             "Noguera","Osona","Pallars Jussà","Pallars Sobirà","Pla d'Urgell","Pla de l'Estany","Priorat",
             "Ribera d'Ebre","Ripollès","Segarra","Segrià","Selva","Solsonès","Tarragonès","Terra Alta",
             "Urgell","Vallès Occidental","Vallès Oriental"]

# 2. Preprocessing data <a class="anchor" id="2"></a>

## 2.1. Dropping NaNs <a class="anchor" id="2.1"></a>

In [4]:
rescue_actions.dropna(axis=0, inplace=True)
protection_plans.dropna(axis=0, inplace=True)
inc_112.dropna(axis=0, inplace=True)
calls_112.dropna(axis=0, inplace=True)

In [5]:
rescue_actions = rescue_actions.reset_index().drop("index",axis=1)
protection_plans = protection_plans.reset_index().drop("index",axis=1)
inc_112 = inc_112.reset_index().drop("index",axis=1)
calls_112 = calls_112.reset_index().drop("index",axis=1)

# 2.2. Cleaning dataset <a class="anchor" id="2.2"></a>

In [6]:
protection_plans.columns

Index(['Tipus pla', 'Número d'inscripció', 'Nom del pla', 'Estat',
       'Data d'homologació', 'Últim esdeveniment',
       'Data de l'últim esdeveniment', 'Ine5', 'Municipi', 'Comarca',
       'Servei territorial', 'Vegueria', 'Província'],
      dtype='object')

In [7]:
dropped_cols = ["Número d'inscripció","Ine5","Vegueria","Servei territorial"]

protection_plans = protection_plans.drop(dropped_cols,axis=1)

In [8]:
for i in range(0,protection_plans.shape[0]):
    protection_plans["Nom del pla"][i] = protection_plans["Nom del pla"][i].split(" ")
    
    if protection_plans["Nom del pla"][i][0] == "PAM" or protection_plans["Nom del pla"][i][0] == "Pam":
        protection_plans["Nom del pla"][i] =  f"PAM {protection_plans['Nom del pla'][i][1]}"
        
    if protection_plans["Nom del pla"][i][0] == "PBEM":
        protection_plans["Nom del pla"][i] = "PBEM"
        
    if protection_plans["Nom del pla"][i][0] == "PA":
        protection_plans["Nom del pla"][i] = "PA PROCICAT"
        
    if protection_plans["Nom del pla"][i][0] == "PEM":
        protection_plans["Nom del pla"][i] = "PEM"
        
    if protection_plans["Nom del pla"][i][0] == "Específic":
        protection_plans["Nom del pla"][i] = "Específic"

In [9]:
tipus_pla = protection_plans["Tipus pla"].unique()
nom_pla = protection_plans["Nom del pla"].unique()
esdeveniment = protection_plans["Últim esdeveniment"].unique()

### Columns

1. __Tipus pla__: T: per als plans territorials. E: per als plans especials. F: per als plans específics. C: per als plans d’actuació. Codi del municipi

2. __Nom pla__: PAM (INFOCAT, SISMICAT, VENTCAT, NEUCAT, INUNCAT, TRANSCAT', PLASEQCAT, CAMCAT, RADCAT, AEROCAT, ALLAUCAT), Específic, PBEM, PEM, PA PROCICAT, PENTA

## 2.3. Datetime <a class="anchor" id="2.3"></a>

In [10]:
protection_plans["Data d'homologació"] = pd.to_datetime(protection_plans["Data d'homologació"], format='%d/%m/%Y')
protection_plans["Data de l'últim esdeveniment"] = pd.to_datetime(protection_plans["Data de l'últim esdeveniment"], format='%d/%m/%Y')


## 2.4. Selecting data <a class="anchor" id="2.4"></a>

In [11]:
protection_plans = protection_plans[(protection_plans["Data d'homologació"].dt.year >= 2018) & (protection_plans["Data d'homologació"].dt.year <= 2022)]
protection_plans = protection_plans.reset_index().drop("index",axis=1)



In [12]:
protection_plans

,Tipus pla,Nom del pla,Estat,Data d'homologació,Últim esdeveniment,Data de l'últim esdeveniment,Municipi,Comarca,Província
0,E,PAM VENTCAT,Homologat,2022-03-10,Homologació,2022-03-10,Tàrrega,Urgell,Lleida
1,E,PAM INFOCAT,Homologat,2021-12-10,Homologació,2021-12-10,Colera,Alt Empordà,Girona
2,E,PAM NEUCAT,Homologat,2022-10-06,Homologació,2022-10-06,Avià,Berguedà,Barcelona
3,E,PAM NEUCAT,Homologat pendent de revisió,2019-03-21,Homologació,2019-03-21,Sant Quirze del Vallès,Vallès Occidental,Barcelona
4,E,PAM VENTCAT,Homologat pendent de revisió,2019-03-21,Homologació,2019-03-21,Sant Quirze del Vallès,Vallès Occidental,Barcelona
...,...,...,...,...,...,...,...,...,...
2758,T,PBEM,Homologat pendent de revisió,2018-12-05,Homologació,2018-12-05,Gósol,Berguedà,Lleida
2759,E,PAM NEUCAT,Homologat pendent de revisió,2019-03-21,Homologació,2019-03-21,Espinelves,Osona,Girona
2760,E,PAM SISMICAT,Homologat pendent de revisió,2019-03-21,Homologació,2019-03-21,Espinelves,Osona,Girona
2761,E,PAM NEUCAT,Homologat pendent de revisió,2019-03-21,Actualització,2019-09-19,Viladrau,Osona,Girona


In [13]:
#rescue_actions = rescue_actions[(rescue_actions["ANY"] >= 2018) & (rescue_actions["ANY"] <= 2022)]
#protection_plans = protection_plans[protection_plans["Data d'homologació"] >= 
#calls_112 = calls_112[(calls_112["ANY"] >= 2018) & (calls_112["ANY"] <= 2022)]
#inc_112 = inc_112[(inc_112["ANY"] >= 2018) & (inc_112["ANY"] <= 2022)]


# 4. Trial plots <a class="anchor" id="3"></a>

In [14]:
cat_map = gpd.read_file(data_dir + "divisions-administratives-v2r1-comarques-100000-20220801.shx")
cat_map.rename(columns = {'NOMCOMAR':'NOM_COMARCA'}, inplace = True)
cat_map.head()

DriverError: dades/divisions-administratives-v2r1-comarques-100000-20220801.shx: No such file or directory

## 3.1 Number of protection plans per Catalonia regions <a class="anchor" id="3.1"></a>

In [ ]:
nplans_provincia = protection_plans.groupby('Província').size().reset_index(name='# plans')
nplans_comarca = protection_plans.groupby('Comarca').size().reset_index(name='# plans')
nplans_municpi = protection_plans.groupby('Municipi').size().reset_index(name='# plans')

### 3.1.1. Províncies <a class="anchor" id="3.1.1"></a>

### 3.1.2. Comarques <a class="anchor" id="3.1.2"></a>

In [ ]:
cat_comarca = gpd.read_file(data_dir + "divisions-administratives-v2r1-comarques-100000-20220801.shx")
cat_comarca.rename(columns = {'NOMCOMAR':'NOM_COMARCA'}, inplace = True)
cat_comarca.head()

In [ ]:
nplans_comarca_merge = nplans_comarca.merge(cat_comarca, how='left', on='NOM_COMARCA')
nplans_comarca_merge_gpd = gpd.GeoDataFrame(nplans_comarca_merge, geometry="geometry")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
nplans_comarca_merge_gpd.plot(column = '# plans',
                                  ax=ax,
                                  cmap='Blues',
                                  legend=True,
                                  #legend_kwds={'orientation': "horizontal", 'pad': 0.02})
#plt.savefig('figures/total_num_2018_2022.pdf')
plt.show()

### 3.1.3. Comarques <a class="anchor" id="3.1.3"></a>